# Labeled Point Data

In this notebook we're going to implement a custom tile generator to serve custom data

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import h5py
import numpy as np
import pandas as pd

### Load data

In this example we show a dataset on image-based spatial transcriptomics by [Wang et al., 2018.](https://doi.org/10.1038/s41598-018-22297-7)

In [ ]:
df = pd.read_csv('data/spatial_gene_expression.csv.gz')

print('Number of points: {}'.format(len(df)))
df.head()

### Custom tileset API

To generate and serve custom tiles we need a `tileset_info()` and a `tiles()` function as demonstrated below.

In [ ]:
from higlass import Tileset
from clodius.tiles.format import format_dense_tile
from clodius.tiles.utils import tile_bounds


def dfdensity(df, max_zoom=5, **kwargs):
    """
    Create an "on-the-fly" point density tileset from a dataframe of (x, y) points.
    To be rendered as a heatmap track.
    
    """
    data = df.reindex(columns=['x', 'y']).values

    tsinfo = {
        'tile_size': 256,
        'min_pos': [df['x'].min(), df['y'].min()],
        'max_pos': [df['x'].max(), df['y'].max()],
        'max_width': max(
            df['x'].max() - df['x'].min(),
            df['y'].max() - df['y'].min()
        ),
        'max_zoom': max_zoom,
        'mirror_tiles': 'false'
    }
    
    def tileset_info():
        return tsinfo
            
    def _get_tile(z, x, y):
        extent = tile_bounds(tsinfo, z, x, y)
        
        # get all the points within the extent
        points = data[
            (data[:, 0] > extent[0]) &
            (data[:, 0] < extent[2]) &
            (data[:, 1] > extent[1]) &
            (data[:, 1] < extent[3])
        ]
        
        # Generate a 2D histogram
        hist, _, _ = np.histogram2d(
            points[:, 0],
            points[:, 1],
            bins=256
        )
        
        # Set empty bins to `nan` to make them transparent
        hist[hist == 0.] = np.nan
        
        return hist.T
    
    def tiles(tile_ids):
        tiles = []
        
        for tile_id in tile_ids:
            # decompose the tile zoom and location
            _, z, x, y = tile_id.split('.')
            
            # generate the tile
            data = _get_tile(int(z), int(x), int(y))
            
            # format the tile response
            tiles.append((tile_id, format_dense_tile(data)))
    
        return tiles

    return Tileset(
        tileset_info=tileset_info,
        tiles=tiles,
        **kwargs
    )

Let's visualize the data as usual

In [ ]:
from higlass import Track, View, display

all_genes_tileset = dfdensity(df, name='Wang et al., 2018. All Genes')

track_config = {
    'track_type': 'heatmap',
    'position': 'center',
    'height': 600,
    'options': {
        'colorRange': ['#ffbb33', '#e5001c', 'black'],
        'backgroundColor': 'white',
    }
}

widget, server, viewconf = display([
    View([
        Track(tileset=all_genes_tileset, **track_config),
    ]),
])

widget

Because we implemented a custom tileset API we can add any kind of customization like picking which channels (i.e., genes) we actually want to visualize.

In [ ]:
## Important change ##########
tilesets = {
    'all_genes': dfdensity(
        df, 
        name='Wang et al., 2018. All Genes'
    ),
    'eno1': dfdensity(
        df[df['gene'] == 'ENO1'], 
        name='Wang et al., 2018. ENO1 Only'
    ),
    'notch2': dfdensity(
        df[df['gene'] == 'NOTCH2'], 
        name='Wang et al., 2018. NOTCH2 Only'
    ),
}

tracks = {
    gene: Track(tileset=ts, **track_config) for gene, ts in tilesets.items()
}
##############################

view_options = {
    'y': 0,
    'width': 4,
}

widget, server, viewconf = display([
    View(x=0, tracks=[tracks['all_genes']], **view_options),
    View(x=4, tracks=[tracks['eno1']], **view_options),
    View(x=8, tracks=[tracks['notch2']], **view_options),
])

widget

**Plasma vs nuclear transcribed RNAs**

In the following closeup we can see that ENO1 is primarily transcriped in the cell body (outside the nucleus) while NOTCH2 is transcribed within the nucleus)

In [ ]:
from copy import deepcopy

closeup_view_options = deepcopy(view_options)
closeup_view_options['initialXDomain'] = [-333, -209]
closeup_view_options['initialYDomain'] = [-202, -30]

closeup_tracks = {
    gene: track.change_options(maxZoom=2) for gene, track in tracks.items()
}

widget, server, viewconf = display([
    View(x=0, tracks=[closeup_tracks['all_genes']], **closeup_view_options),
    View(x=4, tracks=[closeup_tracks['eno1']], **closeup_view_options),
    View(x=8, tracks=[closeup_tracks['notch2']], **closeup_view_options),
])

widget

### Tileset Response from Server

Most HiGlass tracks expect the data to be base64 encoded numpy arrays (`dense`) with a the data type (`dtype`), length `size`, and precalculated min and max values (`min_value` and `max_value`) for efficient value scaling.

In [ ]:
response = server.tiles(tilesets['all_genes'].uuid, 0, 0, 0)
print(
    f'dtype: {response["dtype"]}', 
    f'tile_size: {response["size"]}',
    f'min_value: {response["min_value"]}' ,
    f'max_value: {response["max_value"]}' ,
    f'dense: {response["dense"][:20]}...',
    sep='\n'
)

### View config

To ultimately visualize the data, HiGlass' viewer requires a view configuration telling HiGlass the what, how, and where.

In [ ]:
viewconf

☝️ This dictionary ultimately tells the JavaScript library what dataset to visualize where and how.